## Download the title, image, summary, and file for each PDF 

In [1]:
import os
import csv
import time
from selenium import webdriver
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from unidecode import unidecode

In [2]:
# Load the environment variables
load_dotenv("../.env")

True

In [9]:
def download(url):

    status = False

    try:
        # We need Javascript rendering to see the content
        # Ask Selenium to use Google Chrome as the driver

        # Set WebDriver options (headless mode to run without UI)
        options = Options()
        options.add_argument("--headless=new")
        
        # Ensure you download the right chrome driver from the URL mentioned above for your OS
        # Create a folder named 'chromedriver' and store the chromedriver in it 
        
        # chromedriver.exe will work only on Windows
        chromedriver_directory = os.path.join(os.path.dirname(os.getcwd()), "chromedriver", "chromedriver.exe")
        service = Service(chromedriver_directory)

        # Load the driver from Google Chrome
        driver = webdriver.Chrome(options = options, service = service)

        # Make a GET request
        driver.get(url)

        # Sleep for 10 seconds to ensure the page is completely loaded before proceeding
        time.sleep(10)

        # Fetch the source code of the webpage so we can process it
        page_source_code = driver.page_source

        # Call BeautifulSoup to parse the HTML content
        soup = BeautifulSoup(page_source_code, "html.parser")

        # Extract the PDF download url
        download_url = soup.find('a', class_="content-asset--primary").get("href")

        # Extract the book cover image
        cover_image = soup.find('img', class_="article-cover").get("src").split('?')[0]
        
        # Extract the overview (which will be used as summary)
        overview_content = soup.find_all('div', class_='article__paragraph')
        overview = ""

        for div in overview_content:
            paragraphs = div.find_all('p')
            for para in paragraphs:
                overview += unidecode(str(para.get_text()).strip().replace("\n", ""))
    
    except Exception as exception:
        print(exception)
    
    finally:
        # Stop the webdriver
        driver.quit()
    
    return status

In [10]:
download("https://rpc.cfainstitute.org/research/foundation/2024/beyond-active-and-passive-investing")

/-/media/images/book-cover/cover-beyond-active-and-passive.jpg


False